# Spotify Mood Classifier Testing

In [5]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [6]:
# Authenticate using Spotify API
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Example: Fetch a song by its name and get its audio features
results = sp.search(q='mumei', type='track', limit=1)
track_id = results['tracks']['items'][0]['id']
audio_features = sp.audio_features(track_id)

# Print out the audio features for the song

print(audio_features[0])
results['tracks']['items'][0]['artists'][0]['name']

{'danceability': 0.66, 'energy': 0.656, 'key': 0, 'loudness': -6.746, 'mode': 1, 'speechiness': 0.0281, 'acousticness': 0.0365, 'instrumentalness': 0.00011, 'liveness': 0.39, 'valence': 0.741, 'tempo': 95.987, 'type': 'audio_features', 'id': '7tmkved6SCLlMJGXZhpMJ8', 'uri': 'spotify:track:7tmkved6SCLlMJGXZhpMJ8', 'track_href': 'https://api.spotify.com/v1/tracks/7tmkved6SCLlMJGXZhpMJ8', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7tmkved6SCLlMJGXZhpMJ8', 'duration_ms': 252500, 'time_signature': 4}


'Nanashi Mumei'

In [14]:
playlist_id = '5irzXdNeeKc0Dg3UK4Ww6n'

# Get the tracks in the playlist
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

tracks = get_playlist_tracks(playlist_id)
type(tracks[0])


dict

In [20]:
song_id = tracks[0]['track']['id']
song_id

'7vzSI6FJdJEu17dY2cng8V'

In [21]:
song1 = sp.audio_features(song_id)
song1

[{'danceability': 0.63,
  'energy': 0.207,
  'key': 4,
  'loudness': -13.429,
  'mode': 0,
  'speechiness': 0.0334,
  'acousticness': 0.736,
  'instrumentalness': 2.34e-06,
  'liveness': 0.13,
  'valence': 0.331,
  'tempo': 123.968,
  'type': 'audio_features',
  'id': '7vzSI6FJdJEu17dY2cng8V',
  'uri': 'spotify:track:7vzSI6FJdJEu17dY2cng8V',
  'track_href': 'https://api.spotify.com/v1/tracks/7vzSI6FJdJEu17dY2cng8V',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7vzSI6FJdJEu17dY2cng8V',
  'duration_ms': 125755,
  'time_signature': 4}]